In [1]:
import requests
import json
from time import sleep

In [2]:
# quiet warnings for local testing
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Get Access Token from Visiting Site

#### TODO add POST /token path to return a valid access token 
    based on email and username  

In [334]:
ACCESS_TOKEN ="AglelDrxP41EozEGrxQzOn35vVk1mDgQKjzQ8Jp948mKBayaD6u8Cm4glPwyNvwDbrE760ekpQ61gvIJ7bpDNiy0bdtea23HJy8l"
AC = "?access_token=" + ACCESS_TOKEN
REFRESH_TOKEN = "Ag8bzYrgxJQ4YllpD7E4a3D9jngw9jYg8elkovjN0MKeoyEz0dUaU76nGbv4kl0Yw38nOBoXz5m52z2EDK2DnM1aYV0ok"
RE = "?refresh_token=" + REFRESH_TOKEN

In [335]:
ACCESS = AC

In [12]:
# example options, set custom landing page, delete in 6 minutes
options = {
        "_target": 'http://example.com',
        "_status": 'reserved',
        "ttl": '360'
    } 

# Data Catalog Test

Landing Page : "https://localhost:8080/dc/landing/ark:/99999/fk4RatDC"

In [13]:
dc = {
    "@context": "http://schema.org/",
    "@id": "ark:/99999/fk4RatDC",
    "@type": "DataCatalog",
    "identifier": "ark:/99999/fk4RatDC",
    "name": "Rat Genome Database",
    "url": "http://rgd.mcw.edu/"
}

# with options 
dc_options = {
    'metadata': dc,
    'options': options
}

### Example Put Request

In [42]:
dc_response = requests.put(
    url = "https://localhost/dc/put"+ ACCESS,
    data = json.dumps(dc),
    verify = False
)
json.loads(dc_response.content)

{'api': {'messsage': 'error: bad request - identifier already exists',
  'status': 400},
 'cache': {'created': {'guid': 'ark:/99999/fk4RatDC',
   'name': 'Rat Genome Database',
   'type': 'DataCatalog',
   'url': 'http://rgd.mcw.edu/'}}}

## Example Put with options

Will create an identifier, but also will delete it in 6 minutes

In [17]:
# put with options
options_response = requests.put(
    url = "https://localhost/dc/put"+ ACCESS,
    data = json.dumps(dc_options),
    verify = False
)
options_response.content

b'{"api": {"status": 400, "messsage": "error: bad request - identifier already exists"}, "cache": {"created": {"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}}}'

Test if its still there

In [16]:
dc_get_response = requests.get(
    url = "https://localhost/dc/get/ark:/99999/fk4RatDC" + ACCESS,
    verify = False
)
dc_get_response.content

b'{"success": "ark:/99999/fk4RatDC", "name": "Rat Genome Database", "url": "http://rgd.mcw.edu/", "identifier": "ark:/99999/fk4RatDC", "@context": "http://schema.org/", "@id": "ark:/99999/fk4RatDC", "@type": "DataCatalog"}'

Wait 6 minutes and test again

In [18]:
sleep(360)
dc_get_response = requests.get(
    url = "https://localhost/dc/get/ark:/99999/fk4RatDC" + ACCESS,
    verify = False
)
dc_get_response.content

KeyboardInterrupt: 

### Retrieve the identifier metadata from the cache

Properties about the Data Catalog itself, not members

In [35]:
# get from cache
cache_get_response = requests.get(
    url = "https://localhost/cache/get/ark:/99999/fk4RatDC"+ ACCESS,
    verify = False
)
json.loads(cache_get_response.content.decode('utf-8'))

### Delete the DC from the cache

In [34]:
# delete from cache
cache_delete = requests.delete(
    url = "https://localhost/cache/delete/ark:/99999/fk4RatDC"+ACCESS,
    verify = False
)
json.loads(cache_delete.content.decode('utf-8'))

In [23]:
# import from ezid to neo4j
cache_import = requests.get(
    url = "https://localhost/dc/import/ark:/99999/fk4RatDC"+ ACCESS,
    verify = False
)
json.loads(cache_import.content.decode('utf-8'))

{'cache': {'imported': {'guid': 'ark:/99999/fk4RatDC',
   'name': 'Rat Genome Database',
   'type': 'DataCatalog',
   'url': 'http://rgd.mcw.edu/'}}}

### Delete from both cache and external API

In [597]:
dc_delete = requests.delete(
    url = "https://localhost:8080/dc/delete/ark:/99999/fk4RatDC"+ACCESS,
    verify = False
)
dc_delete.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"cache": {"metadata": {"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}, "message": "Removed from cache"}, "api": {"status_code": 200, "message": "success: ark:/99999/fk4RatDC"}}'

## ARK test

Landing Page https://localhost:8080/ark/landing/ark:/99999/fk4TestDepArk

In [24]:
ark = {
  "@context": "http://schema.org",
  "@id": "ark:/99999/fk4TestDepArk",
  "@type": "CreativeWork",
    "includedInDataCatalog": "ark:/99999/fk4RatDC",
  "identifier": [
   "ark:/88120/r8059v",
   {
      "@type": "PropertyValue",
      "name": "sha-256",
      "value": "cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f"
    }
  ],
  "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v",
  "contentUrl": {"aws": "http://bd2k.ini.usc.edu/assets/all-hands-meeting/minid_v0.1_Nov_2015.pdf"},
  "name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1",
  "author": {
    "@id": "http://orcid.org/0000-0003-2129-5269",
    "@type": "Person",
    "name": "Ian Foster"
  },
  "dateCreated": "2015-11-10T04:44:44.387671Z"
}

ark_options = {
    'metadata': ark,
    'options': options
}


## Put Ark 

In [52]:
local_ark = requests.put(
    url = "https://localhost/ark/put"+ACCESS,
    data = json.dumps(ark),
    verify = False
)
json.loads(local_ark.content.decode('utf-8'))

{'api': {'messsage': 'success: ark:/99999/fk4TestDepArk', 'status': 201},
 'cache': {'created': {'author': 'Ian Foster',
   'dateCreated': '2015-11-10T04:44:44.387671Z',
   'guid': 'ark:/99999/fk4TestDepArk',
   'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
   'type': 'Dataset',
   'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}}}

## Put with Options

In [666]:
local_ark = requests.put(
    url = "https://localhost/ark/put"+ACCESS,
    data = json.dumps(ark_options),
    verify = False
)
json.loads(local_ark.content.decode('utf-8'))

b'{"api": {"status": 201, "messsage": "success: ark:/99999/fk4TestDepArk"}, "cache": {"created": {"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}}}'

In [45]:
# get from api
api_response = requests.get(
    url = "https://localhost/ark/get/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(api_response.content)

{'@context': 'http://schema.org',
 '@id': 'ark:/99999/fk4TestDepArk',
 '@type': 'CreativeWork',
 'author': {'id': 'http://orcid.org/0000-0003-2129-5269',
  'name': 'Ian Foster',
  'type': 'Person'},
 'dateCreated': '2015-11-10T04:44:44.387671Z',
 'identifier': ['ark:/88120/r8059v',
  {'name': 'sha-256',
   'type': 'PropertyValue',
   'value': 'cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f'}],
 'includedInDataCatalog': 'ark:/99999/fk4RatDC',
 'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'success': 'ark:/99999/fk4TestDepArk',
 'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}

In [46]:
# get from cache
cache_get_response = requests.get(
    url = "https://localhost/cache/get/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(cache_get_response.content)

{'author': 'Ian Foster',
 'dateCreated': '2015-11-10T04:44:44.387671Z',
 'guid': 'ark:/99999/fk4TestDepArk',
 'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'type': 'Dataset',
 'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}

In [47]:
# delete from cache
cache_delete = requests.delete(
    url = "https://localhost/cache/delete/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(cache_delete.content)

{'author': 'Ian Foster',
 'dateCreated': '2015-11-10T04:44:44.387671Z',
 'guid': 'ark:/99999/fk4TestDepArk',
 'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'type': 'Dataset',
 'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}

In [59]:
# import from ezid to neo4j
cache_import = requests.get(
    url = "https://localhost/ark/import/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
cache_import.content

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>\n'

ERROR when self.data.get('identifier') is none

In [53]:
# delete from all
ark_delete = requests.delete(
    url = "https://localhost/ark/delete/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
ark_delete.content

b'{"cache": {"metadata": {"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}, "message": "Removed from cache"}, "api": {"status_code": 200, "message": "success: ark:/99999/fk4TestDepArk"}}'

## DOI Test

In [418]:
doi_dc = {
    "@context": "http://schema.org",
    "@type": "DataCatalog",
    "@id": "10.5072/test9999schemaorgtest",
    "identifier": "10.5072/test9999schemaorgtest",
    "additionalType": "Data dictionary",
    "name": "GTEx Public Files",
    "description": "A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
  
    "author": {
        "@type": "Organization",
        "name": "The GTEx Consortium"
    },
  "keywords": "gtex, annotation, phenotype, gene regulation, transcriptomics",
  "datePublished": "2017",
  "version": "v7",
  "url": "https://www.gtexportal.org/home/datasets",
  "contentSize": "5.4 Mb",
  "funder": {
   "@type": "Organization",
   "@id": "doi:/10.13039/100000050",
   "name": "National Heart, Lung, and Blood Institute"
  }
}

In [448]:
doi_dataset = {
    "@context": "http://schema.org",
    "@type": "Dataset",
    "@id": "10.5072/test9999datatest",
    "identifier": [
        "10.5072/test9999datatest",
        {
          "@type": "PropertyValue",
          "name": "sha-256",
          "value": "cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f"
        }
                  ],
    "includedInDataCatalog": "10.5072/test9999schemaorgtest",
    "name": "Download File 1",
    "description": "example download file",
    "keywords": "gtex, annotation, phenotype, gene regulation, transcriptomics",
    "datePublished": "2017",
    "version": "v7",
    "url": "https://www.gtexportal.org/home/datasets",
    "contentSize": "5.4 Mb",
    "fileFormat": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    "contentUrl": ["aws://examplebucket", "gpc://examplebucket"]
}

In [231]:
payload = {
    
    '_profile': 'schema_org',
    '_status': '_reserved',
    '_target': 'https://example.org/',
    'schema_org':  doi
}

In [232]:
doi['@id']

'doi:10.5072/test9999schemaorgtest'

In [273]:
ezapi_get = requests.get(
url = "https://ez.test.datacite.org/id/"+doi['@id']
)

In [218]:
ingestAnvl(ezapi_get.content.decode('utf-8'))

{'_created': '',
 '_datacenter': 'DATACITE.DCPPC',
 '_export': 'yes',
 '_profile': 'datacite',
 '_status': 'reserved',
 '_target': 'https://example.org/',
 '_updated': '1529419918',
 'datacite': '<?xml version="1.0" encoding="UTF-8"?>%0A<resource xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://datacite.org/schema/kernel-4" xsi:schemaLocation="http://datacite.org/schema/kernel-4 http://schema.datacite.org/meta/kernel-4/metadata.xsd">%0A  <identifier identifierType="DOI">10.5072/TEST9999SCHEMAORGTEST</identifier>%0A  <creators/>%0A  <titles>%0A    <title/>%0A  </titles>%0A  <publisher/>%0A  <publicationYear/>%0A  <resourceType resourceTypeGeneral="Text">error</resourceType>%0A  <dates/>%0A  <version/>%0A  <rightsList>%0A    <rights/>%0A  </rightsList>%0A  <descriptions>%0A    <description descriptionType="Abstract"/>%0A  </descriptions>%0A</resource>%0A',
 'success': 'doi:10.5072/test9999schemaorgtest'}

In [373]:
doi['@id']

'10.5072/test9999schemaorgtest'

In [444]:
put_dc = requests.put(
    url = "https://localhost/doi/put"+ ACCESS,
    data = json.dumps(doi_dc),
    verify = False
) 
put_dc.content

In [449]:
put_dataset = requests.put(
    url = "https://localhost/doi/put"+ ACCESS,
    data = json.dumps(doi_dataset),
    verify = False
) 
put_dataset.content

b'<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body bgcolor="white">\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx/1.13.12</center>\r\n</body>\r\n</html>\r\n'

In [417]:
# retrieve from api
get_response = requests.get(
    url = "https://localhost/doi/get/10.5072/test9999schemaorgtest"+ACCESS,
    verify = False
)

get_response.content

b'{"@context": "https://schema.org", "keywords": "gtex, annotation, phenotype, gene regulation, transcriptomics", "author": {"name": "The GTEx Consortium"}, "description": "A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt", "funder": {"name": "National Heart, Lung, and Blood Institute", "@type": "Organization", "@id": "doi:/10.13039/100000050"}, "@type": "DataCatalog", "url": "https://www.gtexportal.org/home/datasets", "includedInDataCatalog": "ark:/99999/fk4RatDC", "name": "GTEx Public Files", "version": "v7", "datePublished": "v7"}'

In [279]:
"https://localhost/doi/get/10.5072/test9999schemaorgtest"+ACCESS

'https://localhost/doi/get/10.5072/test9999schemaorgtest?token=Ag8bzYrgxJQ4YllpD7E4a3D9jngw9jYg8elkovjN0MKeoyEz0dUaU76nGbv4kl0Yw38nOBoXz5m52z2EDK2DnM1aYV0ok'

Landing Page
https://localhost/doi/landing/10.5072/test9999schemaorgtest


In [258]:
# check what you get from ez.datacite
api_get = requests.get(
    url = "https://mds.test.datacite.org/metadata/doi:10.5072/test9999schemaorgtest",
    auth = requests.auth.HTTPBasicAuth('DATACITE.DCPPC', 'Player&Chemo+segment')
)

In [259]:
print(api_get.content.decode('utf-8'))

DOI is unknown to MDS


In [121]:
# test put with different keys
test_body = b'_status: reserver\nMYkey: test'
api_put = requests.put(
    url = "https://ez.test.datacite.org/id/doi:10.5072/test9999randomkeytest",
    data = test_body,
    auth = requests.auth.HTTPBasicAuth('DATACITE.DCPPC', 'Player&Chemo+segment')
)

In [122]:
api_put.content

b'error: no _profile provided'

In [381]:
# retrieve from doi 
get_cache = requests.get(
    url = "https://localhost/cache/get/doi:10.5072/test9999randomkeytest"+ACCESS,
    verify = False
)
json.loads(get_cache.content)

In [345]:
# delete from all
del_all = requests.delete(
    url = "https://localhost/doi/delete/10.5072/test9999test"+ACCESS,
    verify = False
)

In [346]:
del_all.content

b'{"cache": {"metadata": null, "message": "Removed from cache"}, "api": {"status_code": 404, "message": "DOI doesn\'t exist"}}'

In [ ]:
# encode into xml?

In [458]:
# why is postNeo for Doi failing 
from neo4j.v1 import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687", auth=None)

In [519]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        node = tx.run(
            "MATCH (parent:dataCatalog) WHERE parent.guid = $parent "
            "CREATE (node:Doi {guid: $guid, name: $name}) "
            "CREATE (node)-[parentRel:includedIn]->(parent) "
            "RETURN properties(node) ",
            parent = test_doi.get('includedInDataCatalog'),
            guid = test_doi.get('@id'),
            name = test_doi.get('name')
        )

In [520]:
node

In [521]:
#node_data = node.data()
node_data = node.single()

In [522]:
node_data.value()

{'guid': 'doi:0.5072/0000-03VC',
 'name': 'A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt'}

In [525]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        query = tx.run(
            "MATCH (node:Doi) WHERE node.guid = $guid "
            "RETURN properties(node) ",
            guid = test_doi.get('@id')
        )

In [526]:
query.single().value()

{'guid': 'doi:0.5072/0000-03VC',
 'name': 'A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt'}

In [517]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        parent = tx.run(
            "MATCH (parent:dataCatalog) WHERE parent.guid = $parent "
            "RETURN properties(parent) ",
            parent = test_doi.get('includedInDataCatalog'),
        )

In [518]:
parent.single().value()

{'guid': 'ark:/99999/fk4RatDC',
 'name': 'Rat Genome Database',
 'type': 'DataCatalog',
 'url': 'http://rgd.mcw.edu/'}

In [511]:
test_doi.get('includedInDataCatalog')

'ark:/99999/fk4RatDC'

In [527]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        clear = tx.run(
            "MATCH (node:Doi) WHERE node.guid = $guid "
            "DETACH DELETE node ",
            guid = test_doi.get('@id')
        )

In [528]:
clear.data()

[]

In [290]:
from functools import wraps

In [300]:
def execute(f):
    for elem in f:
        print(elem)

In [305]:
listIter = iter([{'key':'val'}, {'key2':'val2'}])

In [308]:
next(listIter)

{'key': 'val'}

In [306]:
dictIter = iter([{'key':'val'}])

In [307]:
next(dictIter)

{'key': 'val'}

In [312]:
myDict = {'key':'val'}

In [315]:
myDict = [{'key':'val'}, {'key2':'val2'}]

In [317]:
next(iter([myDict]))

[{'key': 'val'}, {'key2': 'val2'}]

In [321]:
list(filter(lambda x: isinstance(x, dict), ["hi", {} ]))

TypeError: 'list' object is not an iterator

In [299]:
execute(iter({'mykey':'myval'}))

mykey


In [ ]:
execute(iter)

In [322]:
content = ['awsloc', 'gpcloc']

In [323]:
list(filter(lambda x: re.match('aws', x), content))

['awsloc']

In [324]:
from os import urandom

In [325]:
urandom(16)

b'\xc5\x89\x01)\xecC\xe2\x00L\xd3\xc2\x82+\xec\xb1r'

In [326]:
key = b'\xc5\x89\x01)\xecC\xe2\x00L\xd3\xc2\x82+\xec\xb1r'

In [329]:
from Crypto.Cipher import AES
from Crypto import Random

In [330]:
Random.new().read(AES.block_size)

b'\x1a\xe6\xd6\x95\xf0e\x10eb$\x81\xad\x8c\xd7;\xf1'